<a href="https://colab.research.google.com/github/dllochini/ai-dataset-agent/blob/main/notebooks/simple_data_analysing_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas matplotlib seaborn openai

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import gradio as gr

from google.colab import files, userdata
from openai import OpenAI

client = OpenAI(
    api_key=userdata.get("GROQ_API_KEY"),
    base_url="https://api.groq.com/openai/v1"
)

In [3]:
print("Upload your dataset (CSV):")
uploaded = files.upload()
filename = list(uploaded.keys())[0]
df = pd.read_csv(filename)

print("Dataset loaded successfully!")
display(df.head())

Upload your dataset (CSV):


Saving student_data.csv to student_data (3).csv
Dataset loaded successfully!


,student_id,name,age,gender,math_score,english_score,science_score,attendance_percentage
0,1,Alice,20,Female,85,78,92,95
1,2,Bob,21,Male,72,65,70,88
2,3,Charlie,19,Male,90,88,85,97
3,4,Diana,22,Female,60,75,68,80
4,5,Edward,20,Male,95,91,89,99


In [36]:
SYSTEM_PROMPT = """
You are a dataset analysis AI agent.

You operate strictly in the following loop:

Thought -> Action -> PAUSE -> Observation -> Thought -> ... -> Answer

GENERAL RULES :

1. You must either:
   - Call exactly ONE Action and then output PAUSE
   OR
   - Output a final Answer.

2. After calling an Action, you MUST format it exactly as:

Action: <action_name>[: <column_name_if_required>]
PAUSE

3. After PAUSE, you will receive:
Observation: <tool_output>

4. After receiving an Observation:
   - You MUST continue reasoning.
   - If the observation already contains all information required to answer the user's question,
     you MUST immediately output:

Answer: <final result>

   - Do NOT call additional actions if the answer can already be produced.
   - Do NOT recompute values that are already present in the Observation.
   - Do NOT repeat the same action unless absolutely necessary.

5. Never output an empty message.
6. Never output PAUSE unless you are calling an Action.
7. Never output Observation yourself.
8. Use exact column names when required.
9. Only perform actions directly necessary to answer the question.
10. If a single action provides enough information, immediately produce Answer.

SPECIAL RULES :

• If the user asks for an overview:
  - Only call dataset_overview.
  - After receiving the observation, immediately produce Answer.

• If the user asks for a statistical summary:
  - Only call statistical_summary.
  - After receiving the observation, immediately produce Answer.
  - Do NOT call column_mean or other actions afterward.

AVAILABLE ACTIONS:

dataset_overview
statistical_summary
missing_values
duplicate_count
column_mean: <column_name>
column_min: <column_name>
column_max: <column_name>
value_counts: <column_name>
correlation_matrix
number_of_rows
number_of_columns
plot_numeric_columns
"""

In [5]:
def dataset_overview(df, column=None):
    return {
        "rows": df.shape[0],
        "columns": df.shape[1],
        "column_names": df.columns.tolist(),
        "dtypes": df.dtypes.astype(str).to_dict()
    }

def statistical_summary(df, column=None):
    return df.describe(include='number').to_dict()

def missing_values(df, column=None):
    return df.isnull().sum().to_dict()

def duplicate_count(df, column=None):
    return int(df.duplicated().sum())

def column_mean(df, column=None):
    if column is None or column not in df.columns:
        return "Column not found."
    return df[column].mean()

def column_min(df, column=None):
    if column is None or column not in df.columns:
        return "Column not found."
    return df[column].min()

def column_max(df, column=None):
    if column is None or column not in df.columns:
        return "Column not found."
    return df[column].max()

def value_counts(df, column=None):
    if column is None or column not in df.columns:
        return "Column not found."
    return df[column].value_counts().to_dict()

def correlation_matrix(df, column=None):
    return df.corr(numeric_only=True)

def number_of_rows(df, column=None):
    return df.shape[0]

def number_of_columns(df, column=None):
    return df.shape[1]

def plot_numeric_columns(df, column=None, save_dir="plots"):
    os.makedirs(save_dir, exist_ok=True)
    numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
    saved_paths = []
    for col in numeric_cols:
        plt.figure()
        sns.histplot(df[col], kde=True)
        plt.title(f"Distribution of {col}")
        path = os.path.join(save_dir, f"{col}.png")
        plt.savefig(path)
        plt.close()
        saved_paths.append(path)
    return saved_paths

KNOWN_ACTIONS = {
    "dataset_overview": dataset_overview,
    "statistical_summary": statistical_summary,
    "missing_values": missing_values,
    "duplicate_count": duplicate_count,
    "column_mean": column_mean,
    "column_min": column_min,
    "column_max": column_max,
    "value_counts": value_counts,
    "correlation_matrix": correlation_matrix,
    "number_of_rows": number_of_rows,
    "number_of_columns": number_of_columns,
    "plot_numeric_columns": plot_numeric_columns,
}

In [6]:
class Agent:
    def __init__(self, system_prompt):
        self.messages = [{"role": "system", "content": system_prompt}]

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})

        response = client.chat.completions.create(
            model="llama-3.1-8b-instant",
            messages=self.messages,
        )

        content = response.choices[0].message.content
        self.messages.append({"role": "assistant", "content": content})

        return content


In [38]:
bot = Agent(system_prompt=SYSTEM_PROMPT)

Example 1

In [39]:
que = "give me a overview of the datatset"

print(bot(que))

Action: dataset_overview
PAUSE


In [40]:
result = dataset_overview(df)
print('function result :', result)
next_prompt = "Observation: {}".format(result)
print(next_prompt)

function result : {'rows': 10, 'columns': 8, 'column_names': ['student_id', 'name', 'age', 'gender', 'math_score', 'english_score', 'science_score', 'attendance_percentage'], 'dtypes': {'student_id': 'int64', 'name': 'object', 'age': 'int64', 'gender': 'object', 'math_score': 'int64', 'english_score': 'int64', 'science_score': 'int64', 'attendance_percentage': 'int64'}}
Observation: {'rows': 10, 'columns': 8, 'column_names': ['student_id', 'name', 'age', 'gender', 'math_score', 'english_score', 'science_score', 'attendance_percentage'], 'dtypes': {'student_id': 'int64', 'name': 'object', 'age': 'int64', 'gender': 'object', 'math_score': 'int64', 'english_score': 'int64', 'science_score': 'int64', 'attendance_percentage': 'int64'}}


In [41]:
bot(next_prompt)

"The dataset has 10 rows and 8 columns, including categorical data types (object) for 'name' and 'gender' and numerical data types for the rest of the columns. \n\nAnswer: The dataset has 10 rows and 8 columns, with categorical data types for 'name' and 'gender' and numerical data types for the rest of the columns."

Example 2

In [49]:
que = "What is the minimum math_score?"

print(bot(que))

You already know this answer as it's in the previous Observation. 

Answer: The minimum math_score is 60.


In [50]:
result = column_min(df,"math_score")
print(result)
next_prompt = "Observation: {}".format(result)
print(next_prompt)

60
Observation: 60


In [51]:
bot(next_prompt)

"You're repeating observations now. Since I already know the answer from a previous observation and the current observation doesn't contain any new information, the answer remains the same.\n\nAnswer: The minimum math_score is 60."

Example 3

In [46]:
que = "give me a statistical overview of the datatset"

print(bot(que))

Action: statistical_summary
PAUSE


In [47]:
result = statistical_summary(df)
print(result)
next_prompt = "Observation: {}".format(result)
print(next_prompt)

{'student_id': {'count': 10.0, 'mean': 5.5, 'std': 3.0276503540974917, 'min': 1.0, '25%': 3.25, '50%': 5.5, '75%': 7.75, 'max': 10.0}, 'age': {'count': 10.0, 'mean': 20.7, 'std': 1.3374935098492586, 'min': 19.0, '25%': 20.0, '50%': 20.5, '75%': 21.75, 'max': 23.0}, 'math_score': {'count': 10.0, 'mean': 80.0, 'std': 11.8227652339788, 'min': 60.0, '25%': 72.5, '50%': 81.5, '75%': 89.5, 'max': 95.0}, 'english_score': {'count': 10.0, 'mean': 79.6, 'std': 10.035492569431314, 'min': 65.0, '25%': 71.25, '50%': 80.0, '75%': 87.25, 'max': 94.0}, 'science_score': {'count': 10.0, 'mean': 82.2, 'std': 9.986657765906136, 'min': 68.0, '25%': 73.0, '50%': 84.5, '75%': 89.75, 'max': 96.0}, 'attendance_percentage': {'count': 10.0, 'mean': 91.2, 'std': 6.250333324444918, 'min': 80.0, '25%': 87.25, '50%': 91.5, '75%': 96.5, 'max': 99.0}}
Observation: {'student_id': {'count': 10.0, 'mean': 5.5, 'std': 3.0276503540974917, 'min': 1.0, '25%': 3.25, '50%': 5.5, '75%': 7.75, 'max': 10.0}, 'age': {'count': 10.0

In [48]:
bot(next_prompt)

'Answer: The dataset has a count of 10, with mean values of 5.5 for student_id, 20.7 for age, 80.0 for math_score, 79.6 for english_score, 82.2 for science_score, and 91.2 for attendance_percentage. The standard deviations are 3.03 for student_id, 1.34 for age, 11.82 for math_score, 10.04 for english_score, 9.99 for science_score, and 6.25 for attendance_percentage.'

Query Loop

In [33]:
import re
import json

action_re = re.compile(r"Action:\s*(\w+)(?::\s*(.*))?")

def query_loop(question, df, max_iters=8):
    agent = Agent(SYSTEM_PROMPT)
    reasoning_steps = []
    next_input = question

    for _ in range(max_iters):

        output = agent(next_input)
        reasoning_steps.append(output)

        if not output.strip():
            return "Model returned empty response.", reasoning_steps, None

        if "Answer:" in output:
            final_answer = output.split("Answer:", 1)[1].strip()
            return final_answer, reasoning_steps, None

        match = action_re.search(output)

        if match:

            if "PAUSE" not in output:
                return "Protocol error: Model must output PAUSE after Action.", reasoning_steps, None

            action_name = match.group(1).strip().lower()
            action_input = match.group(2).strip() if match.group(2) else None

            if action_name not in KNOWN_ACTIONS:
              message = f"Unable to perform action '{action_name}' — this action is not supported."
              reasoning_steps.append(message)
              next_input = f"Observation: {message}"
              continue

            observation = KNOWN_ACTIONS[action_name](df, action_input)

            try:
                serialized = json.dumps(observation, default=str)
            except:
                serialized = str(observation)

            reasoning_steps.append(f"Observation (JSON): {serialized}")

            next_input = f"Observation: {serialized}"

            continue

        return output, reasoning_steps, None

    return "Agent stopped after too many reasoning steps.", reasoning_steps, None

In [35]:
que = ["Give me an overview of this dataset.",
       "How many rows and columns are there?",
       "List all column names and their data types.",
       "Tell me which columns are numeric and which are categorical.",
       "Provide a statistical summary for all numeric columns.",
       "Give me the mean, min, max, and standard deviation of all columns.",
       "Show descriptive statistics of this dataset.",
       "What is the mean age of students?",
       "What is the maximum math_score?",
       "How many unique values are in the gender column?",
       "Check which columns have missing values.",
       "Are there any duplicate rows?",
       "Give me the correlation matrix for numeric columns.",
       "Plot the distribution of all numeric columns.",
       "Which student has the highest science_score and what is their attendance_percentage?",
       "Compare math_score and english_score correlation and mean values.",
       "Give me the mean of all columns.",
       "How many students have attendance_percentage above 90?"
       ]

query_loop(que[17], df)

('Agent stopped after too many reasoning steps.',
 ['Action: value_counts: attendance_percentage\nPAUSE',
  'Observation (JSON): {"95": 1, "88": 1, "97": 1, "80": 1, "99": 1, "90": 1, "85": 1, "93": 1, "87": 1, "98": 1}',
  'Action: value_counts: attendance_percentage\nPAUSE',
  'Observation (JSON): {"95": 1, "88": 1, "97": 1, "80": 1, "99": 1, "90": 1, "85": 1, "93": 1, "87": 1, "98": 1}',
  'Since this action only produces the counts for each value of the attendance_percentage, it does not provide the final answer to the question "How many students have attendance_percentage above 90?". I will proceed with a different action.\n\nAction: dataset_overview\nPAUSE',
  'Observation (JSON): {"rows": 10, "columns": 8, "column_names": ["student_id", "name", "age", "gender", "math_score", "english_score", "science_score", "attendance_percentage"], "dtypes": {"student_id": "int64", "name": "object", "age": "int64", "gender": "object", "math_score": "int64", "english_score": "int64", "science_s